In [1]:
import numpy as np

In [70]:
from typing import List,Tuple

In [151]:
def one_indexed_vertices_to_zero_indexed(
    edges:List[Tuple[int,int]]
)->List[Tuple[int,int]]:
    return [(v1-1,v2-1) for (v1,v2) in edges]
def make_pr_matrix(
    edges:List[Tuple[int,int]],n_vertices:int
)->List[List[int]]:
    part1=np.array(
        [[1 for _ in range(n_vertices)] for _ in range(n_vertices)]
        ,dtype=float
    )
    part2=np.array(
        [[0 for _ in range(n_vertices)] for _ in range(n_vertices)]
        ,dtype=float
    )
    out_degrees={v:0 for v in range(n_vertices)}
    for (v1,_) in edges:
        out_degrees[v1]+=1
    for (v1,v2) in edges:
        part2[v2,v1]=1/out_degrees[v1]
    for v,out_degree in out_degrees.items():
        if out_degree==0:
            part2[:,v]=1/n_vertices
    return .15/n_vertices*part1+.85*part2

In [74]:
make_pr_matrix([(1,2),(2,3),(1,3)],5)

array([[0.2  , 0.03 , 0.03 , 0.2  , 0.2  ],
       [0.2  , 0.03 , 0.03 , 0.2  , 0.2  ],
       [0.2  , 0.455, 0.03 , 0.2  , 0.2  ],
       [0.2  , 0.455, 0.88 , 0.2  , 0.2  ],
       [0.2  , 0.03 , 0.03 , 0.2  , 0.2  ]])

First internet in notes.

In [78]:
internet1_pr_matrix=make_pr_matrix(
    one_indexed_vertices_to_zero_indexed([
        (1,2),
        (1,3),
        (2,3),
        (3,4),
        (4,3)
    ])
    ,4
)

Second internet in notes.

In [79]:
internet2_pr_matrix=make_pr_matrix(
    one_indexed_vertices_to_zero_indexed([
        (1,2),
        (1,3),
        (2,3),
        (2,4),
        (4,2)
    ])
    ,4
)

In [81]:
internet1_pr_matrix

array([[0.0375, 0.0375, 0.0375, 0.0375],
       [0.4625, 0.0375, 0.0375, 0.0375],
       [0.4625, 0.8875, 0.0375, 0.8875],
       [0.0375, 0.0375, 0.8875, 0.0375]])

In [152]:
def get_probability_eigenvector(pr_matrix):
    eigenpairs=np.linalg.eig(pr_matrix)
    ix=[
        i for i in range(len(eigenpairs.eigenvalues)) 
        if np.isclose(eigenpairs.eigenvalues.tolist()[i],1)
    ][0]
    return eigenpairs.eigenvectors[ix]/sum(eigenpairs.eigenvectors[ix])

In [119]:
np.linalg.eig(internet1_pr_matrix).eigenvectors[0]

array([ 5.80013853e-02+0.00000000e+00j, -7.87146130e-17+4.09712372e-09j,
       -7.87146130e-17-4.09712372e-09j, -3.93326756e-17+0.00000000e+00j])

In [118]:
get_probability_eigenvector(internet1_pr_matrix)

array([ 1.00000000e+00+0.00000000e+00j, -1.35711609e-15+7.06383769e-08j,
       -1.35711609e-15-7.06383769e-08j, -6.78133383e-16+0.00000000e+00j])

In [97]:
np.isclose(np.linalg.eig(internet1_pr_matrix).eigenvalues.tolist()[0],1)

np.True_

In [121]:
online_solution=[4440/51853,6327/51853,55780/51853,1]

In [123]:
np.array(online_solution)/sum(online_solution)

array([0.0375    , 0.0534375 , 0.47111486, 0.43794764])

https://www.emathhelp.net/calculators/linear-algebra/eigenvalue-and-eigenvector-calculator/?i=%5B%5B0.0375%2C0.0375%2C0.0375%2C0.0375%5D%2C%5B0.4625%2C0.0375%2C0.0375%2C0.0375%5D%2C%5B0.4625%2C0.8875%2C0.0375%2C0.8875%5D%2C%5B0.0375%2C0.0375%2C0.8875%2C0.0375%5D%5D

This is right, then why did numpy give the wrong answer?

In [155]:
get_eigvec=lambda i:[
    round(float(z.real),3) 
    for z in 
    np.linalg.eig(internet1_pr_matrix).eigenvectors[0]
    /sum(np.linalg.eig(internet1_pr_matrix).eigenvectors[i])
]

In [156]:
get_eigvec(0)

[1.0, -0.0, -0.0, -0.0]

In [157]:
get_eigvec(1)

[0.039, -0.0, -0.0, -0.0]

In [158]:
get_eigvec(2)

[2.689, -0.0, -0.0, -0.0]

In [159]:
get_eigvec(3)

[-1.951, 0.0, 0.0, 0.0]

None of these are it.

In [160]:
[
    round(float(z.real),4) 
    for z in 
    np.linalg.eig(internet1_pr_matrix).eigenvectors.T[0]
    /sum(np.linalg.eig(internet1_pr_matrix).eigenvectors.T[0])
]

[0.0375, 0.0534, 0.4711, 0.4379]

Got it.

Ok so numpy's eigenvector collection is transposed also.

In [180]:
def get_probability_eigenvector(pr_matrix:List[List[float]]):
    eigenpairs=np.linalg.eig(pr_matrix)
    ix=[
        i for i in range(len(eigenpairs.eigenvalues)) 
        if np.isclose(eigenpairs.eigenvalues.tolist()[i],1)
    ][0]
    return [round(float(x.real),4) for x in eigenpairs.eigenvectors[:,ix]/sum(eigenpairs.eigenvectors[:,ix])]

In [170]:
get_probability_eigenvector(internet1_pr_matrix)

[0.03749999999999999,
 0.05343749999999988,
 0.47111486486486487,
 0.4379476351351353]

Exercise 7.1.

In [171]:
get_probability_eigenvector(
    make_pr_matrix(
        one_indexed_vertices_to_zero_indexed([
            (1,2)
        ])
        ,2
    )
)

[0.3508771929824561, 0.6491228070175439]

In [175]:
make_pr_matrix(
    one_indexed_vertices_to_zero_indexed([
        (1,2)
    ])
    ,2
)

array([[0.075, 0.5  ],
       [0.925, 0.5  ]])

Exercise 7.2.

In [178]:
get_probability_eigenvector(
    make_pr_matrix(
        one_indexed_vertices_to_zero_indexed([
            (1,6),
            (2,1),
            (2,4),
            (2,6),
            (3,2),
            (5,6),
            (6,4)
        ])
        ,6
    )
)

[0.1114, 0.1352, 0.0731, 0.3393, 0.0731, 0.2681]

Exercise 7.4.

In [179]:
get_probability_eigenvector(
    make_pr_matrix(
        one_indexed_vertices_to_zero_indexed([
            (2,1),
            (3,2),
            (4,5),
            (4,6),
            (5,6)
        ])
        ,6
    )
)

[0.2454, 0.1765, 0.0954, 0.0954, 0.1359, 0.2515]

Exercise 7.5.

In [182]:
e7_5_matrix=make_pr_matrix(
    one_indexed_vertices_to_zero_indexed([
        (2,1),
        (3,2),
        (4,5),
        (4,6),
        (5,6)
    ])
    ,6
)

In [205]:
def get_decimal_places_equal(f1:float,f2:float)->int:
    s1,s2=str(f1),str(f2)
    if s1[:s1.index('.')]!=s2[:s2.index('.')]:
        return 0
    s1chop=s1[s1.index('.')+1:]
    s2chop=s2[s2.index('.')+1:]
    i=0
    while i<min(len(s1chop),len(s2chop)) and s1chop[i]==s2chop[i]:
        i+=1
    return i

In [229]:
def iterations_until_numerical_convergence(
    pr_matrix:np.array,vector:np.array
)->int:
    n=len(vector)
    steps=0
    prev=vector
    vector=pr_matrix@vector
    decimals_equal=min([
        get_decimal_places_equal(prev,vector) 
        for i in range(n)
    ])
    while decimals_equal<4:
        prev=vector
        vector=pr_matrix@vector
        decimals_equal=min([
            get_decimal_places_equal(prev[i],vector[i]) 
            for i in range(n)
        ])
        steps+=1
        print(f'{steps=}\n{prev=}, \n{vector=},')
        print(f'{get_decimal_places_equal(prev,vector)=}\n')
    return steps

In [230]:
iterations_until_numerical_convergence(
    e7_5_matrix,
    np.array([1,0,0,0,0,0],dtype=float).T
)

steps=1
prev=array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667]), 
vector=array([0.21388889, 0.21388889, 0.07222222, 0.07222222, 0.14305556,
       0.28472222]),
get_decimal_places_equal(prev,vector)=0

steps=2
prev=array([0.21388889, 0.21388889, 0.07222222, 0.07222222, 0.14305556,
       0.28472222]), 
vector=array([0.27744213, 0.15702546, 0.09563657, 0.09563657, 0.12633102,
       0.24792824]),
get_decimal_places_equal(prev,vector)=1

steps=3
prev=array([0.27744213, 0.15702546, 0.09563657, 0.09563657, 0.12633102,
       0.24792824]), 
vector=array([0.23289911, 0.18071856, 0.09942747, 0.09942747, 0.14007301,
       0.24745438]),
get_decimal_places_equal(prev,vector)=1

steps=4
prev=array([0.23289911, 0.18071856, 0.09942747, 0.09942747, 0.14007301,
       0.24745438]), 
vector=array([0.24666085, 0.17756343, 0.09305008, 0.09305008, 0.13530675,
       0.25436881]),
get_decimal_places_equal(prev,vector)=1

steps=5
prev=array([0.24666085, 0.17756343, 0.09

12

In [186]:
e7_5_matrix.round(3)

array([[0.167, 0.875, 0.025, 0.025, 0.025, 0.167],
       [0.167, 0.025, 0.875, 0.025, 0.025, 0.167],
       [0.167, 0.025, 0.025, 0.025, 0.025, 0.167],
       [0.167, 0.025, 0.025, 0.025, 0.025, 0.167],
       [0.167, 0.025, 0.025, 0.45 , 0.025, 0.167],
       [0.167, 0.025, 0.025, 0.45 , 0.875, 0.167]])

Exercise 7.7.

In [231]:
def make_pr_matrix_exercise_7_7(
    edges:List[Tuple[int,int]],
    n_vertices:int,
    p:float#Fraction of weight to put on the constant matrix term.
)->List[List[int]]:
    part1=np.array(
        [[1 for _ in range(n_vertices)] for _ in range(n_vertices)]
        ,dtype=float
    )
    part2=np.array(
        [[0 for _ in range(n_vertices)] for _ in range(n_vertices)]
        ,dtype=float
    )
    out_degrees={v:0 for v in range(n_vertices)}
    for (v1,_) in edges:
        out_degrees[v1]+=1
    for (v1,v2) in edges:
        part2[v2,v1]=1/out_degrees[v1]
    for v,out_degree in out_degrees.items():
        if out_degree==0:
            part2[:,v]=1/n_vertices
    return p/n_vertices*part1+(1-p)*part2

In [236]:
e7_7_eigvecs=[get_probability_eigenvector(make_pr_matrix_exercise_7_7(
    one_indexed_vertices_to_zero_indexed([
        (1,2),
        (1,3),
        (2,3),
        (3,1)
    ])
    ,3
    ,p
)) for p in [.85,.15,.6,.4]]

In [237]:
e7_7_eigvecs

[[0.3366, 0.3086, 0.3549],
 [0.3878, 0.2148, 0.3974],
 [0.3514, 0.2703, 0.3784],
 [0.367, 0.2434, 0.3895]]

In [244]:
def order(li:List[float])->List[int]:
    li_sorted=sorted(li)
    return [li_sorted.index(li[ix]) for ix in range(len(li))]

In [246]:
[order(li) for li in e7_7_eigvecs]

[[1, 0, 2], [1, 0, 2], [1, 0, 2], [1, 0, 2]]

In [251]:
get_probability_eigenvector(make_pr_matrix_exercise_7_7(
    one_indexed_vertices_to_zero_indexed([
        (1,2),
        (1,3),
        (2,3),
        (3,1)
    ])
    ,3
    ,1
))

[0.3333, 0.3333, 0.3333]

Exercise 7.8.

In [253]:
get_probability_eigenvector(make_pr_matrix(
    one_indexed_vertices_to_zero_indexed([
        (1,2),
        (1,5),
        (1,10),
        (2,5),
        (2,1),
        (3,5),
        (4,7),
        (4,9),
        (5,4),
        (5,7),
        (6,9),
        (7,3),
        (7,10),
        (8,2),
        (8,3),
        (9,1),
        (9,6),
        (9,7),
        (9,8),
        (10,1),
        (10,3)
    ])
    ,10
))

[0.1042,
 0.0584,
 0.1408,
 0.0954,
 0.1891,
 0.0327,
 0.1536,
 0.0327,
 0.0833,
 0.1098]

In [254]:
print([0.1042,
 0.0584,
 0.1408,
 0.0954,
 0.1891,
 0.0327,
 0.1536,
 0.0327,
 0.0833,
 0.1098])

[0.1042, 0.0584, 0.1408, 0.0954, 0.1891, 0.0327, 0.1536, 0.0327, 0.0833, 0.1098]


In [255]:
get_probability_eigenvector(make_pr_matrix(
    one_indexed_vertices_to_zero_indexed([
        (1,2),
        (2,3),
        (3,1)
    ])
    ,3
))

[0.3333, 0.3333, 0.3333]

In [256]:
get_probability_eigenvector(make_pr_matrix(
    one_indexed_vertices_to_zero_indexed([
        (1,2),
        (2,3)
    ])
    ,3
))

[0.1844, 0.3412, 0.4744]

Exercise 7.10.

In [262]:
def line(n):
    return [(z,z+1)
            for z in range(n)
            if z!=(n-1)]

In [274]:
def line(n:int)->List[List[float]]:
    return make_pr_matrix([
            (z,z+1)
            for z in range(n)
            if z!=n-1
        ]
        ,n
    )

In [275]:
line(5)

array([[0.03, 0.03, 0.03, 0.03, 0.2 ],
       [0.88, 0.03, 0.03, 0.03, 0.2 ],
       [0.03, 0.88, 0.03, 0.03, 0.2 ],
       [0.03, 0.03, 0.88, 0.03, 0.2 ],
       [0.03, 0.03, 0.03, 0.88, 0.2 ]])